# Importation des modules

In [89]:
###### modules pour le chargement des données depuis le XML ######
import glob
from lxml import etree
from preTraitements.xml import get_X_Y_from_root
from preTraitements.xml import get_tree_root_from_file

###### modules pour la classification ######

# modèles
from sklearn.svm import LinearSVC, SVC

# vectorisation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import StandardScaler

# création de nos transformers
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction import DictVectorizer # créer nos propres transformer

# recherche des meilleurs hyperparamètres
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

# résultats
from sklearn.metrics import classification_report

# sauvegarde des modèles
from joblib import dump, load

###### modules pour la visualisation ######
import matplotlib.pyplot as plt
import pandas as pd

###### miscellaneous ######
from typing import List # typage des fonctions
import numpy as np
import re
from collections import namedtuple

# Pré-traitement des données

## Chargement des données dans le bon format

In [90]:
tree_train, root_train = get_tree_root_from_file("./corpus/train_deft09_parlement_appr.xml/deft09_parlement_appr_fr.xml")
X_train, y_train = get_X_Y_from_root(root_train)

tree_test, root_test = get_tree_root_from_file("./corpus/deft09_parlement_test.xml/deft09_parlement_test_fr.xml")
X_test, y_test = get_X_Y_from_root(root_test) # y_test est vide : pas accès aux résultats

## Nettoyage des données

enlever les majuscules et la ponctuation

In [91]:
pattern_clean = re.compile(r"[^ \w]") # pattern à utiliser pour nettoyer les données

def clean(data): # TODO: améliorer la fonction
    """TODO: écrire la docstring

    Args:
        data (_type_): _description_

    Returns:
        _type_: _description_
    """
    global pattern_clean
    return re.sub(pattern_clean, "", data).lower()

X_train_clean = [clean(x) for x in X_train]
X_test_clean = [clean(x) for x in X_test]


In [92]:
X_train_sample = X_train_clean[:500]
y_train_sample = y_train[:500]
X_test_sample = X_test_clean[:500]

# Extraction des features

`token_patternstr or None, default=r”(?u)\b\w\w+\b` pour countvectorizer en mettant la regex de grobol

## GloVe

## Word2vec

## FastText

## Transformers personnalisés

(nombre de mots par énoncé, nombre de phrases par énoncés)

# Création des PipeLine

on crée d'abord toutes les pipelines qu'on veut

on aura une param_grid par pipeline

In [93]:
# named tuple Classifieur : 
# - pipeline
# - param_grid pour le grid_search
Classifieur = namedtuple('Classifieur', ["pipeline", "param_grid"]) #TODO : instancier le nom du classifieur pour que ça soit facile à récupérer après

## `LinearSVC()`

```python
    "svm__dual":(True,False), # algorithme de décision (incompatible avec penalty l1)
```

In [94]:
# liste de tuples nommés Classifieur(pipeline, param_grid)
liste_clfs = []

pipeline_svm = Pipeline([
        ('ngram_tf_idf', Pipeline([
          ('counts', CountVectorizer()),
          ('tf_idf', TfidfTransformer())
        ])),
  #('standard', StandardScaler(with_mean=False)),
  ('svm', SVC(kernel="linear",probability=True))])

param_grid_svm = {
    "ngram_tf_idf__counts__max_df": (0.2, 0.4, 0.6, 0.8, 1.0), # fréquence maximum auquel il s'arrête
    "ngram_tf_idf__counts__min_df": (1,5, 10), # fréquence minimum
    #"ngram_tf_idf__counts__ngram_range": ((1, 1), (1, 2),(2,2)),  # unigrams or bigrams
    #"ngram_tf_idf__counts__stop_words":('french',None), #TODO: pas de french : utiliser la liste de spacy ou nltk
    "ngram_tf_idf__tf_idf__use_idf":(True,False)
    #"clf__penalty": ('l1','l2'),
    #"svm__C":(0.1,0.5,1.0,1.5) # complexité du modèle
    }

svm = Classifieur(pipeline_svm, param_grid_svm)
liste_clfs.append(svm)

# Recherche de la meilleure pipeline

on a toutes nos pipelines dans une liste et on fait une boucle pour obtenir le dataframe avec les cv_results_ de toutes les pipeline

on concatène tous les df et on sort_values sur "mean_test_score"

In [95]:
best_models = []

for (pipeline, param_grid) in liste_clfs:
    grid_search = GridSearchCV(pipeline, param_grid=param_grid)
    estimator = grid_search.fit(X_train_sample, y_train_sample)
    best_models.append(estimator)
    
best_models

[GridSearchCV(estimator=Pipeline(steps=[('ngram_tf_idf',
                                         Pipeline(steps=[('counts',
                                                          CountVectorizer()),
                                                         ('tf_idf',
                                                          TfidfTransformer())])),
                                        ('svm',
                                         SVC(kernel='linear',
                                             probability=True))]),
              param_grid={'ngram_tf_idf__counts__max_df': (0.2, 0.4, 0.6, 0.8,
                                                           1.0),
                          'ngram_tf_idf__counts__min_df': (1, 5, 10),
                          'ngram_tf_idf__tf_idf__use_idf': (True, False)})]

# Visualisation des résultats

- `predict_proba` donne une liste de probabilité
- `clf.classes_` donne les étiquettes des classes (pour qu'on récupère l'ordre)

Les colonnes qu'on veut dans notre dataframe récapitulatif : 

- le nom du classifieur
- `estimator.best_params_` = les paramètres sous forme de dictionnaire du meilleur modèle trouvé avec le gridsearch
- `estimator.best_estimator_` = la pipeline du meilleur modèle
- `estimator.best_score_` = le mean_test_score du meilleur modèle trouvé avec le gridsearch


In [96]:
best_models[0].best_estimator_

Pipeline(steps=[('ngram_tf_idf',
                 Pipeline(steps=[('counts',
                                  CountVectorizer(max_df=0.2, min_df=10)),
                                 ('tf_idf', TfidfTransformer(use_idf=False))])),
                ('svm', SVC(kernel='linear', probability=True))])